# Construyendo con SQL

Exploraremos cómo construir una base de datos y familiarizarnos con la sintaxis SQL.

[SQLlite](https://www.sqlite.org/index.html) es una implementación minimalista de un sistema de gestión de base de datos con soporte de transacciones. Veréis que se almacena como un fichero en nuestro directorio local.

[Referencia](https://www.sqlitetutorial.net/)

In [1]:
import sqlite3
import pandas as pd

# Connect to the DB
conn = sqlite3.connect("DB_EJERCICIO.db")
c = conn.cursor()

Indicaremos que queremos que funcione como una sistema operacional, manteniendo la integridad.

In [2]:
c.execute("""PRAGMA foreign_keys = ON;""");

Y crearemos nuestra primera tabla:

- Nombre: Categories
- Campos: 
    - CategoryID: Entero, clave primaria y autoincrementada
    - CategoryName: Texto
    - Description: Texto

In [3]:
query_create = """
CREATE TABLE Categories
(      
    CategoryID INTEGER PRIMARY KEY AUTOINCREMENT,
    CategoryName TEXT,
    Description TEXT
);
"""

c.execute(query_create);

In [4]:
tablas = c.execute("""SELECT name FROM sqlite_master WHERE type='table';""").fetchall()
for tabla in tablas:
    print(tabla)

('Categories',)
('sqlite_sequence',)


In [5]:
c.execute("""SELECT * FROM Categories""").fetchall()

[]

Está vacía, podemos insertar nuestro primer dato ahora que existe.

In [6]:
query_insert = """
INSERT INTO Categories VALUES(null,'Beverages','Soft drinks, coffees, teas, beers, and ales');
"""

c.execute(query_insert);

In [7]:
c.execute("""SELECT * FROM Categories""").fetchall()

[(1, 'Beverages', 'Soft drinks, coffees, teas, beers, and ales')]

Si intentamos insertar una fila con el mismo identificador veremos que la base de datos se encarga de darnos un error. Es una operación no permitida.

In [8]:
query = """
INSERT INTO Categories VALUES(1,'Beverages','Soft drinks, coffees, teas, beers, and ales');
"""

c.execute(query);

IntegrityError: UNIQUE constraint failed: Categories.CategoryID

In [9]:
query = """
INSERT INTO Categories VALUES(2,'Condiments','Sweet and savory sauces, relishes, spreads, and seasonings');
INSERT INTO Categories VALUES(3,'Confections','Desserts, candies, and sweet breads');
INSERT INTO Categories VALUES(4,'Dairy Products','Cheeses');
INSERT INTO Categories VALUES(5,'Grains/Cereals','Breads, crackers, pasta, and cereal');
INSERT INTO Categories VALUES(6,'Meat/Poultry','Prepared meats');
INSERT INTO Categories VALUES(7,'Produce','Dried fruit and bean curd');
INSERT INTO Categories VALUES(8,'Seafood','Seaweed and fish');
"""

c.executescript(query);

In [10]:
tuplas = c.execute("""SELECT * FROM Categories""").fetchall()
for tupla in tuplas:
    print(tupla)

(1, 'Beverages', 'Soft drinks, coffees, teas, beers, and ales')
(2, 'Condiments', 'Sweet and savory sauces, relishes, spreads, and seasonings')
(3, 'Confections', 'Desserts, candies, and sweet breads')
(4, 'Dairy Products', 'Cheeses')
(5, 'Grains/Cereals', 'Breads, crackers, pasta, and cereal')
(6, 'Meat/Poultry', 'Prepared meats')
(7, 'Produce', 'Dried fruit and bean curd')
(8, 'Seafood', 'Seaweed and fish')


Podemos, eso si, actualizar la información de nuestro sistema empleando la clausula `update` y gracias al identificador solo actualizar el registro que nos atañe.

In [11]:
c.execute("""UPDATE Categories SET CategoryName = 'Drinks' WHERE CategoryID == 1""").fetchall()

[]

In [12]:
tuplas = c.execute("""SELECT * FROM Categories""").fetchall()
for tupla in tuplas:
    print(tupla)

(1, 'Drinks', 'Soft drinks, coffees, teas, beers, and ales')
(2, 'Condiments', 'Sweet and savory sauces, relishes, spreads, and seasonings')
(3, 'Confections', 'Desserts, candies, and sweet breads')
(4, 'Dairy Products', 'Cheeses')
(5, 'Grains/Cereals', 'Breads, crackers, pasta, and cereal')
(6, 'Meat/Poultry', 'Prepared meats')
(7, 'Produce', 'Dried fruit and bean curd')
(8, 'Seafood', 'Seaweed and fish')


O borrar con el comando `DELETE`.

In [13]:
c.execute("""DELETE FROM Categories WHERE CategoryID == 8""").fetchall()

[]

In [14]:
tuplas = c.execute("""SELECT * FROM Categories""").fetchall()
for tupla in tuplas:
    print(tupla)

(1, 'Drinks', 'Soft drinks, coffees, teas, beers, and ales')
(2, 'Condiments', 'Sweet and savory sauces, relishes, spreads, and seasonings')
(3, 'Confections', 'Desserts, candies, and sweet breads')
(4, 'Dairy Products', 'Cheeses')
(5, 'Grains/Cereals', 'Breads, crackers, pasta, and cereal')
(6, 'Meat/Poultry', 'Prepared meats')
(7, 'Produce', 'Dried fruit and bean curd')


Extenderemos el modelo, en este caso empleando dos nuevas estructuras de datos: **proveedores** y **productos**.

In [15]:
query = """
CREATE TABLE Suppliers(
    SupplierID INTEGER PRIMARY KEY AUTOINCREMENT,
    SupplierName TEXT,
    ContactName TEXT,
    Address TEXT,
    City TEXT,
    PostalCode TEXT,
    Country TEXT,
    Phone TEXT
);
"""

c.execute(query);

In [16]:
query = """
INSERT INTO Suppliers VALUES(1,'Exotic Liquid','Charlotte Cooper','49 Gilbert St.','Londona','EC1 4SD','UK','(171) 555-2222');
INSERT INTO Suppliers VALUES(2,'New Orleans Cajun Delights','Shelley Burke','P.O. Box 78934','New Orleans','70117','USA','(100) 555-4822');
INSERT INTO Suppliers VALUES(3,'Grandma Kelly''s Homestead','Regina Murphy','707 Oxford Rd.','Ann Arbor','48104','USA','(313) 555-5735');
"""

c.executescript(query);

In [17]:
query = """
CREATE TABLE Products(
    ProductID INTEGER PRIMARY KEY AUTOINCREMENT,
    ProductName TEXT,
    SupplierID INTEGER,
    CategoryID INTEGER,
    Unit TEXT,
    Price NUMERIC DEFAULT 0,
	FOREIGN KEY (CategoryID) REFERENCES Categories (CategoryID),
	FOREIGN KEY (SupplierID) REFERENCES Suppliers (SupplierID)
);
"""

c.execute(query);

In [ ]:
query = """
INSERT INTO Products VALUES(1,'Chais',1,1,'10 boxes x 20 bags',18.00);
INSERT INTO Products VALUES(2,'Chang',1,1,'24 - 12 oz bottles',19.00);
INSERT INTO Products VALUES(3,'Aniseed Syrup',1,2,'12 - 550 ml bottles',10.00);
INSERT INTO Products VALUES(4,'Chef Anton''s Cajun Seasoning',1,2,'48 - 6 oz jars',22.00);
"""

c.executescript(query);

In [18]:
tablas = c.execute("""SELECT name FROM sqlite_master WHERE type='table';""").fetchall()
for tabla in tablas:
    print(tabla)

('Categories',)
('sqlite_sequence',)
('Suppliers',)
('Products',)


Y podemos utilizar las clausulas `JOIN` que ya vimos para obtener la información desnormalizada que nos es de interés.

In [19]:
import pandas as pd

tuples = c.execute("""SELECT * FROM Categories c JOIN Products p ON c.CategoryID == p.CategoryID""").fetchall()
pd.DataFrame(tuples)

""


Veremos que las restricciones de sistema también afectan a las decisiones de borrado ya que nos impiden dejar el sistema en un estado inconsistente.

In [20]:
c.execute("""DELETE FROM Categories WHERE CategoryID == 1""").fetchall()

[]

In [21]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas y ejecutamos una sentencia
def sql_query(query):

    # Ejecuta la query
    c.execute(query)

    # Almacena los datos de la query 
    ans = c.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in c.description]

    return pd.DataFrame(ans,columns=names)



###### ejemplo para ejecutar el def sql_query
# Nombre de la tabla de la que quieres obtener los campos
nombre_tabla = 'EMPLOYEES'

# Ejecutar la consulta PRAGMA para obtener la información de las columnas
query = f'PRAGMA table_info({nombre_tabla});'
df_columnas = sql_query(query)

# Mostrar el DataFrame con la información de las columnas
print(df_columnas)

Empty DataFrame
Columns: [cid, name, type, notnull, dflt_value, pk]
Index: []


Cerramos la conexión con base de datos.

In [ ]:
c.close()

## Herramientas específicas

Podéis descargaros https://dbeaver.io/ para explorar y disponer de una interfaz alternativa (más expresiva) a la hora de explorar bases de datos relacionales.